In [1]:
from warnings import warn
import numpy as np
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.calculators.emt import EMT
from ase import units
from ase.md.velocitydistribution import PhononHarmonics, MaxwellBoltzmannDistribution
from ase.io import read, Trajectory

In [2]:
atoms = read('Al.in.supercell', 0, 'aims')
fc = np.loadtxt('force_constants_Al.dat')

In [3]:
warn('This will only work after MR 995 is merged to ASE')

/home/knoop/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: This will only work after MR 995 is merged to ASE
  """Entry point for launching an IPython kernel.


## Set Up MD

In [4]:
T = 600 * units.kB

atoms_disp = atoms.copy()
atoms_disp.calc = EMT()

PhononHarmonics(atoms_disp, fc, T, quantum=False, rng=np.random.RandomState(1))

atoms_disp.get_temperature()

614.6308346779284

In [5]:
log_settings = {
    'trajectory': 'md.traj',
    'logfile': 'md.log'
}

In [6]:
md = Langevin(atoms_disp, timestep=4*units.fs, **log_settings, friction=0.02, temperature=T)

## Run MD for some while

In [7]:
for _ in md.irun(steps=5):
    atoms_disp.info.update({'md_dt': md.dt, 'md_nstep': md.nsteps})

In [8]:
md.nsteps

5

In [9]:
with Trajectory(log_settings['trajectory'], 'r') as traj:
    trajectory = [at for at in traj]

for at in trajectory:
    print(at.info)

{'md_dt': 0.39290779153856253, 'md_nstep': 0}
{'md_dt': 0.39290779153856253, 'md_nstep': 1}
{'md_dt': 0.39290779153856253, 'md_nstep': 2}
{'md_dt': 0.39290779153856253, 'md_nstep': 3}
{'md_dt': 0.39290779153856253, 'md_nstep': 4}
{'md_dt': 0.39290779153856253, 'md_nstep': 5}


In [10]:
# starting point for next MD section
    
nstep_start = trajectory[-1].info['md_nstep']

## Run a new MD and attach to old trajectory

In [11]:
atoms_disp = trajectory[-1].copy()
atoms_disp.calc = EMT()

In [12]:
md = VelocityVerlet(atoms_disp, timestep=4*units.fs, **log_settings, append_trajectory=True)
md.nsteps = nstep_start
        
for _ in md.irun(steps=5):
    atoms_disp.info.update({'md_dt': md.dt, 'md_nstep': md.nsteps})

In [13]:
with Trajectory(log_settings['trajectory'], 'r') as traj:
    trajectory = [at for at in traj]

In [14]:
for at in trajectory:
    print(at.info)

{'md_dt': 0.39290779153856253, 'md_nstep': 0}
{'md_dt': 0.39290779153856253, 'md_nstep': 1}
{'md_dt': 0.39290779153856253, 'md_nstep': 2}
{'md_dt': 0.39290779153856253, 'md_nstep': 3}
{'md_dt': 0.39290779153856253, 'md_nstep': 4}
{'md_dt': 0.39290779153856253, 'md_nstep': 5}
{'md_dt': 0.39290779153856253, 'md_nstep': 6}
{'md_dt': 0.39290779153856253, 'md_nstep': 7}
{'md_dt': 0.39290779153856253, 'md_nstep': 8}
{'md_dt': 0.39290779153856253, 'md_nstep': 9}
{'md_dt': 0.39290779153856253, 'md_nstep': 10}
